In [2]:
%cd /home/lcc/projects/MetaHarmonizer

/home/lcc/projects/MetaHarmonizer


/home/lcc/miniconda3/envs/py310/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import pandas as pd
from importlib import reload
from src.models import ontology_mapper_st as om_st
from src.Engine import ontology_mapping_engine as ome
import src.models.ontology_mapper_lm as om_lm
import src.models.ontology_mapper_rag_faiss as om_rag
import src.models.ontology_mapper_bi_encoder as om_bi

reload(om_st)
reload(ome)
reload(om_lm)
reload(om_rag)
reload(om_bi)

/home/lcc/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lcc/miniconda3/envs/py310/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lcc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lcc/nltk_data...
[nltk_data]   Package

<module 'src.models.ontology_mapper_bi_encoder' from '/home/lcc/projects/MetaHarmonizer/src/models/ontology_mapper_bi_encoder.py'>

In [5]:
from src.models.calc_stats import CalcStats

calc = CalcStats()

In [6]:
from src.utils.cleanup_vector_store import cleanup_vector_store

cleanup_vector_store("rag","pubmed-bert","disease")
# cleanup_vector_store("st","mt-sap-bert","disease_sample")

[Success] Table 'rag_pubmed_bert_disease' dropped from src/KnowledgeDb/vector_db.sqlite
[Warning] Index file 'src/KnowledgeDb/faiss_indexes/rag_pubmed-bert_disease.index' does not exist.


Find out mismatch, update the csv files and re-run code

In [ ]:
from src.KnowledgeDb.db_clients.nci_db import NCIDb
import asyncio

# Load data
df = pd.read_csv('data/corpus/cbio_disease/disease_corpus_from_NCIT:C3262.csv')

# Init API
nci_db = NCIDb("07bb3180-6b91-44a7-8fc4-ca051b1bacb2")

# Clean codes
df["clean_code"] = df["obo_id"].str.replace("NCIT:", "", regex=False).str.strip()
codes = df["clean_code"].dropna().unique().tolist()

# Fetch labels
labels = asyncio.run(nci_db.get_labels_by_codes(codes))

# Debug print
print("=== Official NCI code → label ===")
for code, label in labels.items():
    print(f"{code}: {label}")

# Map label
df["official_label"] = df["clean_code"].map(labels)

# Compare
df["label_mismatch"] = (
    df["label"].astype(str).str.strip().str.lower() !=
    df["official_label"].astype(str).str.strip().str.lower()
)

# Output mismatches
mismatches = df[df["label_mismatch"]]
print(f"Found {len(mismatches)} mismatched records.")
print(mismatches[["obo_id", "label", "official_label"]].head())

01/07/2025 02:19:35 AM - INFO - NCIDb: Fetching concept data for 13924 codes in batches of 50
01/07/2025 02:19:39 AM - INFO - NCIDb: Processed batch 1 of 279
01/07/2025 02:19:39 AM - INFO - NCIDb: Processed batch 2 of 279
01/07/2025 02:19:42 AM - INFO - NCIDb: Processed batch 3 of 279
01/07/2025 02:19:48 AM - INFO - NCIDb: Processed batch 5 of 279
01/07/2025 02:19:51 AM - INFO - NCIDb: Processed batch 6 of 279
01/07/2025 02:19:53 AM - INFO - NCIDb: Processed batch 7 of 279
01/07/2025 02:19:53 AM - INFO - NCIDb: Processed batch 4 of 279
01/07/2025 02:19:56 AM - INFO - NCIDb: Processed batch 8 of 279
01/07/2025 02:19:59 AM - INFO - NCIDb: Processed batch 9 of 279
01/07/2025 02:20:04 AM - INFO - NCIDb: Processed batch 11 of 279
01/07/2025 02:20:07 AM - INFO - NCIDb: Processed batch 12 of 279
01/07/2025 02:20:10 AM - INFO - NCIDb: Processed batch 13 of 279
01/07/2025 02:20:10 AM - INFO - NCIDb: Processed batch 10 of 279
01/07/2025 02:20:13 AM - INFO - NCIDb: Processed batch 14 of 279
01/07

In [ ]:
# Convert mismatch flag to string for clarity
df["label_mismatch"] = df["label_mismatch"].map({True: "Yes", False: "No"})

# Save full dataframe with official_label and mismatch flag
df.to_csv("disease_corpus_updated.csv", index=False)

print("Saved updated file as disease_corpus_updated.csv with official_label and label_mismatch columns.")

Saved updated file as disease_corpus_updated.csv with official_label and label_mismatch columns.


In [ ]:
import pandas as pd


df_query = pd.read_csv("data/corpus/cbio_disease/disease_query_updated.csv")
df_corpus = pd.read_csv("data/corpus/cbio_disease/disease_corpus_updated.csv")


query_labels = df_query["curated_ontology"].dropna().str.strip().unique()
corpus_labels = df_corpus["official_label"].dropna().str.strip().unique()


unmatched = [label for label in query_labels if label not in corpus_labels]


print(f"Found {len(unmatched)} unmatched labels:")
for label in unmatched:
    print(label)


Found 5 unmatched labels:
Unclassified Renal Cell Carcinoma
Carcinoid Tumor
Neoplasm
Central Nervous System Melanoma
Skin Metatypical Carcinoma


Find if our queries have the same shortName as EDA/cancer_type_mapping.csv and if they have the same meaning

In [ ]:
import pandas as pd

# read two CSV files
query_df = pd.read_csv("data/corpus/cbio_disease/disease_query_updated.csv")  #  original_value  curated_ontology
mapping_df = pd.read_csv("data/corpus/cancer_type_mapping.csv")  #  shortName  name

# merge: find out where original_value == shortName
merged_df = query_df.merge(mapping_df, left_on="original_value", right_on="shortName", how="inner")

# remain only the necessary columns
output_df = merged_df[["original_value", "curated_ontology", "shortName", "name"]]

In [ ]:
output_df

,original_value,curated_ontology,shortName,name
0,HGSOC,Ovarian High Grade Serous Adenocarcinoma,HGSOC,High-Grade Serous Ovarian Cancer
1,LGSOC,Ovarian Low Grade Serous Adenocarcinoma,LGSOC,Low-Grade Serous Ovarian Cancer
2,PANET,Pancreatic Neuroendocrine Tumor,PANET,Pancreatic Neuroendocrine Tumor
3,PRAD,Prostate Adenocarcinoma,PRAD,Prostate Adenocarcinoma
4,NSCLC,Lung Non-Small Cell Carcinoma,NSCLC,Non-Small Cell Lung Cancer
...,...,...,...,...
61,ACC,Adrenal Cortical Carcinoma,ACC,Adrenocortical Carcinoma
62,PLMESO,Pleural Mesothelial Neoplasm,PLMESO,Pleural Mesothelioma
63,READ,Rectal Adenocarcinoma,READ,Rectal Adenocarcinoma
64,DIPG,Diffuse Intrinsic Pontine Glioma,DIPG,Diffuse Intrinsic Pontine Glioma


In [ ]:
output_df.to_csv("matched_queries_with_mapping.csv", index=False)

For the queries we are using, the shortName can be transferred to name. Need a more robust way for dealing with abbreviations.

apply shortname - name transition

In [5]:
df = pd.read_csv("data/corpus/cbio_disease/disease_query_updated.csv")
large_corpus = pd.read_csv(
    'data/corpus/cbio_disease/disease_corpus_updated.csv')

query_list = df['original_value'].tolist()
small_corpus_list = df['curated_ontology'].tolist()
large_corpus_list = large_corpus['official_label'].tolist()

cura_map = dict(zip(df['original_value'], df['curated_ontology']))

In [6]:
# ST Sapbert
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='mt-sap-bert',
                                      category='disease_sample',
                                      topk=5,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='st',
                                      **other_params)
st_sapbert_result = onto_engine_large.run()

05/07/2025 05:34:26 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
05/07/2025 05:34:26 AM - INFO - OntoMapEngine: Running Ontology Mapping
05/07/2025 05:34:26 AM - INFO - OntoMapEngine: Separating exact and non-exact matches
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replacing shortNames using rule-based name mapping
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replaced: ACC → Adrenocortical Carcinoma
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replaced: ACYC → Adenoid Cystic Carcinoma
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replaced: AML → Acute Myeloid Leukemia
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replaced: ANSC → Anal Squamous Cell Carcinoma
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replaced: APAD → Appendiceal Adenocarcinoma
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replaced: ASPS → Alveolar Soft Part Sarcoma
05/07/2025 05:34:27 AM - INFO - OntoMapEngine: Replaced: ATRT → Atypical Teratoid/Rhabdoid Tumor
05/07/2025 05:34:27 AM - INF

No sentence-transformers model found with name model_cache/mt-sap-bert. Creating a new one with mean pooling.
No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext-mean-token. Creating a new one with mean pooling.


In [7]:
st_sapbert_accuracy_df = calc.calc_accuracy(st_sapbert_result)
print(st_sapbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  75.370729
1  Top 3 Matches  84.655061
2  Top 5 Matches  87.685364


In [8]:
st_sapbert_result

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score,updated_value,top1_accuracy,top3_accuracy,top5_accuracy
0,25-30% DUCTAL COMPONENT,Prostate Ductal Adenocarcinoma,99,2,Ductal Eccrine Adenocarcinoma,0.6203,Oligometastatic Pancreatic Ductal Adenocarcinoma,0.6008,Ductal Eccrine Carcinoma with Abundant Fibromy...,0.5990,Diffuse Intraductal Papillomatosis,0.5958,Intraductal Papilloma,0.5946,25-30% DUCTAL COMPONENT,False,False,False
1,82463 NEUROENDOCRINE CARCINOMA NOS,Neuroendocrine Carcinoma,1,2,Neuroendocrine Carcinoma,0.7585,Cervical Neuroendocrine Carcinoma,0.7151,Advanced Neuroendocrine Carcinoma,0.7141,Grade III Neuroendocrine Carcinoma,0.7064,Gastric Neuroendocrine Carcinoma,0.7034,82463 NEUROENDOCRINE CARCINOMA NOS,True,True,True
2,AB,Thymoma Type AB,99,2,Branchioma,0.6178,Blastoma,0.6003,Adult Acute Basophilic Leukemia,0.5979,Acute Basophilic Leukemia,0.5913,Barrett Adenocarcinoma,0.5850,AB,False,False,False
3,ACANTHOLYTIC,Skin Acantholytic Squamous Cell Carcinoma,99,2,Acantholytic Acanthoma,0.8033,Acanthoma,0.7408,Epidermolytic Acanthoma,0.7229,Clear Cell Acanthoma,0.6689,Large Cell Acanthoma,0.6590,ACANTHOLYTIC,False,False,False
4,ACC,Adrenal Cortical Carcinoma,1,2,Adrenal Cortical Carcinoma,0.9037,Advanced Adrenal Cortical Carcinoma,0.8379,Malignant Adrenal Cortical Neoplasm,0.8115,Metastatic Adrenal Cortical Carcinoma,0.8101,Adrenal Cortical Neoplasm,0.8089,Adrenocortical Carcinoma,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,WILMS' TUMOR,Wilms Tumor,1,2,Wilms Tumor,0.9868,Kidney Wilms Tumor,0.9630,Childhood Kidney Wilms Tumor,0.9016,Prostate Wilms Tumor,0.8785,Ovarian Wilms Tumor,0.8731,WILMS' TUMOR,True,True,True
1547,WILMS' TUMORS,Wilms Tumor,1,2,Wilms Tumor,0.9844,Kidney Wilms Tumor,0.9605,Childhood Kidney Wilms Tumor,0.8999,Prostate Wilms Tumor,0.8818,Ovarian Wilms Tumor,0.8748,WILMS' TUMORS,True,True,True
1548,WITH DUCTAL AND LOBULAR PHENOTYPES,Breast Mixed Ductal and Lobular Carcinoma,1,2,Breast Mixed Ductal and Lobular Carcinoma,0.6855,Diffuse Lipomatosis,0.6514,Diffuse Intraductal Papillomatosis,0.6390,Double Pituitary Neuroendocrine Tumors of Dist...,0.6261,Invasive Breast Ductal Carcinoma and Lobular C...,0.6235,WITH DUCTAL AND LOBULAR PHENOTYPES,True,True,True
1549,WT,Wilms Tumor,1,2,Wilms Tumor,0.9868,Kidney Wilms Tumor,0.9630,Childhood Kidney Wilms Tumor,0.9016,Prostate Wilms Tumor,0.8785,Ovarian Wilms Tumor,0.8731,Wilms' Tumor,True,True,True


In [9]:
st_sapbert_result.to_csv("data/corpus/cbio_disease/st_sapbert_disease_new_result.csv", index=False)

Filtered out columns with less than 30% non-missing values of cBioPortal_all_clinicalData_combined_2023-05-18.rds

And saved the data into non_na_ratio_0.3.csv

In [ ]:
import pandas as pd

filterd_non_na_gt_30pct = "data/non_na_ratio_0.3.csv"
non_na_gt_30pct_df = pd.read_csv(filterd_non_na_gt_30pct)

print(f"Data dimensions: {non_na_gt_30pct_df.shape}")
print("First 5 rows:")
display(non_na_gt_30pct_df.head())

Data dimensions: (189439, 18)
First 5 rows:


,studyId,patientId,SAMPLE_COUNT,sampleId,MUTATION_COUNT,CANCER_TYPE_DETAILED,CANCER_TYPE,ONCOTREE_CODE,TMB_NONSYNONYMOUS,SOMATIC_STATUS,SEX,AGE,FRACTION_GENOME_ALTERED,SAMPLE_TYPE,OS_MONTHS,OS_STATUS,RACE,PRIMARY_SITE
0,acc_tcga,TCGA-OR-A5J1,1.0,TCGA-OR-A5J1-01,39.0,Adrenocortical Carcinoma,Adrenocortical Carcinoma,ACC,1.266667,Matched,Male,58.0,0.0585,Primary,44.51,1:DECEASED,WHITE,NaN
1,acc_tcga,TCGA-OR-A5J2,1.0,TCGA-OR-A5J2-01,81.0,Adrenocortical Carcinoma,Adrenocortical Carcinoma,ACC,2.733333,Matched,Female,44.0,0.4033,Primary,55.09,1:DECEASED,WHITE,NaN
2,acc_tcga,TCGA-OR-A5J3,1.0,TCGA-OR-A5J3-01,69.0,Adrenocortical Carcinoma,Adrenocortical Carcinoma,ACC,2.333333,Matched,Female,23.0,0.4131,Primary,68.69,0:LIVING,WHITE,NaN
3,acc_tcga,TCGA-OR-A5J4,1.0,TCGA-OR-A5J4-01,171.0,Adrenocortical Carcinoma,Adrenocortical Carcinoma,ACC,5.666667,Matched,Female,23.0,0.9259,Primary,13.90,1:DECEASED,WHITE,NaN
4,acc_tcga,TCGA-OR-A5J5,1.0,TCGA-OR-A5J5-01,521.0,Adrenocortical Carcinoma,Adrenocortical Carcinoma,ACC,17.366667,Matched,Male,30.0,0.8015,Primary,11.99,1:DECEASED,WHITE,NaN


In [ ]:
missing_ratio = non_na_gt_30pct_df.isnull().mean()
print("Missing ratio of top 10 columns:")
display(missing_ratio.sort_values(ascending=False).head(10))

Missing ratio of top 10 columns:


PRIMARY_SITE               0.661157
AGE                        0.548567
RACE                       0.477642
SOMATIC_STATUS             0.466430
OS_MONTHS                  0.442433
FRACTION_GENOME_ALTERED    0.432804
OS_STATUS                  0.414872
SAMPLE_TYPE                0.349009
MUTATION_COUNT             0.186018
TMB_NONSYNONYMOUS          0.181473
dtype: float64

#### Columns that not related to ontology mapping for Disease:
sampleId  
SAMPLE_COUNT  
OS_STATUS  
OS_MONTHS
SAMPLE_TYPE
patientId (since we've got studyId)
RACE (maybe some diseases are related to them, optional)  
SAMPLE_TYPE

In [ ]:
null_count = non_na_gt_30pct_df["ONCOTREE_CODE"].isna().sum()
print(f"Number of null ONCOTREE_CODE: {null_count}")

unique_valid_codes = non_na_gt_30pct_df["ONCOTREE_CODE"].dropna().nunique()
print(f"Number of unique valid ONCOTREE_CODE: {unique_valid_codes}")

Number of null ONCOTREE_CODE: 20430
Number of unique valid ONCOTREE_CODE: 589


In [ ]:
print(non_na_gt_30pct_df["ONCOTREE_CODE"].dropna().unique()[:10])
print(non_na_gt_30pct_df["ONCOTREE_CODE"].value_counts(dropna=True).head(10))

['ACC' 'BALL' 'AUL' 'ALL' 'LEUK' 'TALL' 'AML' 'BLL' 'ACBC' 'ACYC']
ONCOTREE_CODE
LUAD     16904
PRAD     15289
IDC      10514
BLCA      5750
BRCA      5746
AML       4827
COAD      4754
PAAD      3841
ALUCA     3575
GBM       3345
Name: count, dtype: int64


In [11]:
import requests

def fetch_oncotree_tree(version="oncotree_latest_stable"):
    url = "http://oncotree.mskcc.org/api/tumorTypes/tree"
    params = {"version": version}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    return resp.json()

def extract_code_name_pairs(tree_dict):
    result = {}

    def dfs(node):
        code = node.get("code")
        name = node.get("name")
        if code and name:
            result[code] = name
        for child in node.get("children", {}).values():
            dfs(child)

    for top_node in tree_dict.values():
        dfs(top_node)

    return result

oncotree_data = fetch_oncotree_tree()

code_to_name = extract_code_name_pairs(oncotree_data)

for code, name in list(code_to_name.items())[:10]:
    print(f"{code}: {name}")

import pandas as pd

df = pd.DataFrame(list(code_to_name.items()), columns=["code", "name"])
df.to_csv("data/corpus/oncotree_code_to_name.csv", index=False)

TISSUE: Tissue
OVARY: Ovary/Fallopian Tube
OOVC: Ovarian Cancer, Other
OCNOS: Ovarian Choriocarcinoma, NOS
HGONEC: High-Grade Neuroendocrine Carcinoma of the Ovary
HGSFT: High-Grade Serous Fallopian Tube Cancer
OVT: Ovarian Epithelial Tumor
EBOV: Endometrioid Borderlin Ovarian Tumor
SOC: Serous Ovarian Cancer
HGSOC: High-Grade Serous Ovarian Cancer


In [15]:
import pandas as pd

df1 = pd.read_csv('data/corpus/cancer_type_mapping.csv')
df1['key'] = df1['shortName'].astype(str).str.strip() + ' - ' + df1['name'].astype(str).str.strip()
set1 = set(df1['key'])

df2 = pd.read_csv('data/corpus/oncotree_code_to_name.csv')
df2['key'] = df2['code'].astype(str).str.strip() + ' - ' + df2['name'].astype(str).str.strip()
set2 = set(df2['key'])

duplicates = set1 & set2

if duplicates:
    print("Duplicate items found in both files:")
    print(f"Total duplicates: {len(duplicates)}")
    for item in sorted(duplicates):
        print(item)
else:
    print("No duplicate items found in both files.")


Duplicate items found in both files:
Total duplicates: 891
AA - Aggressive Angiomyxoma
AASTR - Anaplastic Astrocytoma
ABC - Activated B-cell Type
ABL - Acute Basophilic Leukemia
ACA - Adrenocortical Adenoma
ACBC - Adenoid Cystic Breast Cancer
ACC - Adrenocortical Carcinoma
ACCC - Acinic Cell Carcinoma
ACLG - Adenoid Cystic Carcinoma of the Lacrimal Gland
ACML - Atypical Chronic Myeloid Leukemia, BCR-ABL1-
ACN - Acinar Cell Carcinoma, NOS
ACPG - Craniopharyngioma, Adamantinomatous Type
ACPP - Atypical Choroid Plexus Papilloma
ACRM - Acral Melanoma
ACYC - Adenoid Cystic Carcinoma
ADMA - Adamantinoma
ADNOS - Adenocarcinoma, NOS
ADPA - Aggressive Digital Papillary Adenocarcinoma
ADRENAL_GLAND - Adrenal Gland
AECA - Sweat Gland Carcinoma/Apocrine Eccrine Carcinoma
AFH - Angiomatoid Fibrous Histiocytoma
AFX - Atypical Fibroxanthoma
AGA - Anal Gland Adenocarcinoma
AGNG - Anaplastic Ganglioglioma
AHCD - Alpha Heavy-Chain Disease
AIS - Adenocarcinoma In Situ
AITL - Angioimmunoblastic T-Cell Lym

Use curated_disease for testing matching and comparing. Scratch.

In [ ]:
curated_df = pd.read_csv("data/corpus/cbio_disease/curated_disease.csv")
print(f"Data dimensions: {curated_df.shape}")

Data dimensions: (189439, 30)


/tmp/ipykernel_2378/4091141321.py:1: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  curated_df = pd.read_csv("data/corpus/cbio_disease/curated_disease.csv")


In [ ]:
# filter out null
filtered_df = curated_df.dropna(
    subset=[
        "curated_cancer_type_detailed_ontology_term_id",
        "original_cancer_type_value",
        "curated_cancer_type_ontology_term_id"
    ])

# filter out cases with multiple disease terms (semicolon-separated)
filtered_df = filtered_df[
    ~filtered_df["curated_cancer_type_ontology_term_id"].str.
    contains(";", regex=False)]

# remain those with valid NCIT codes
filtered_df = filtered_df[filtered_df[
    "curated_cancer_type_ontology_term_id"].str.startswith("NCIT:")]

In [ ]:
print(f"Filtered data dimensions: {filtered_df.shape}")

Filtered data dimensions: (181439, 30)


In [ ]:
filtered_df.to_csv("data/corpus/cbio_disease/curated_single_disease_NCIT.csv", index=False)

In [ ]:
are_equal = (filtered_df["curated_cancer_type_ontology_term_id"] == filtered_df["curated_cancer_type_detailed_ontology_term_id"])
all_equal = are_equal.all()

print(
    f"Are all 'curated_cancer_type_ontology_term_id' and 'curated_cancer_type_detailed_ontology_term_id' equal? {all_equal}"
)

diff_df = filtered_df[~are_equal]
print(f"Number of rows with differences: {diff_df.shape[0]}")
display(diff_df.head())

Are all 'curated_cancer_type_ontology_term_id' and 'curated_cancer_type_detailed_ontology_term_id' equal? False
Number of rows with differences: 158334


,curation_id,original_cancer_status_value,curated_cancer_status_source,curated_cancer_status,curated_cancer_status_ontology_term_id,original_disease_value,curated_disease_source,curated_disease,curated_disease_ontology_term_id,original_cancer_subtype_value,...,curated_sample_type,curated_sample_type_ontology_term_id,original_cancer_type_value,curated_cancer_type,curated_cancer_type_ontology_term_id,curated_cancer_type_source,original_cancer_type_detailed_value,curated_cancer_type_detailed_source,curated_cancer_type_detailed,curated_cancer_type_detailed_ontology_term_id
92,all_stjude_2015:SJINF001:SJINF001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LEUKEMIA,Leukemia,NCIT:C3161,CANCER_TYPE,B-CELL ACUTE LYMPHOID LEUKEMIA,CANCER_TYPE_DETAILED,B Acute Lymphoblastic Leukemia,NCIT:C8644
93,all_stjude_2015:SJINF001:SJINF001R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LEUKEMIA,Leukemia,NCIT:C3161,CANCER_TYPE,B-CELL ACUTE LYMPHOID LEUKEMIA,CANCER_TYPE_DETAILED,B Acute Lymphoblastic Leukemia,NCIT:C8644
94,all_stjude_2015:SJINF002:SJINF002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LEUKEMIA,Leukemia,NCIT:C3161,CANCER_TYPE,B-CELL ACUTE LYMPHOID LEUKEMIA,CANCER_TYPE_DETAILED,B Acute Lymphoblastic Leukemia,NCIT:C8644
95,all_stjude_2015:SJINF002:SJINF002R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LEUKEMIA,Leukemia,NCIT:C3161,CANCER_TYPE,B-CELL ACUTE LYMPHOID LEUKEMIA,CANCER_TYPE_DETAILED,B Acute Lymphoblastic Leukemia,NCIT:C8644
96,all_stjude_2015:SJINF003:SJINF003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LEUKEMIA,Leukemia,NCIT:C3161,CANCER_TYPE,B-CELL ACUTE LYMPHOID LEUKEMIA,CANCER_TYPE_DETAILED,B Acute Lymphoblastic Leukemia,NCIT:C8644


so curated_cancer_type_detailed_ontology_term_id is the real ontology term id

In [ ]:
# create a sample DataFrame for testing
unique_df = filtered_df.drop_duplicates(
    subset="curated_cancer_type_ontology_term_id")

available = unique_df.shape[0]
if available < 1000:
    print(
        f"Warning: Only {available} unique disease terms available, returning all."
    )
    sampled_df = unique_df.copy()
else:
    sampled_df = unique_df.sample(n=1000, random_state=42)

print(f"Sampled data dimensions: {sampled_df.shape}")

Sampled data dimensions: (134, 30)


In [ ]:
sampled_df.to_csv("data/corpus/cbio_disease/curated_single_disease_NCIT_sampled.csv",
                   index=False)

In [25]:
from src.utils.update_term_via_code import update_ncit_labels

sample_df_with_official_label = update_ncit_labels(
    input_path="data/corpus/cbio_disease/curated_single_disease_NCIT_sampled.csv",
    ncit_code_column="curated_cancer_type_ontology_term_id",
    label_column="curated_cancer_type",
    save_csv=True,
    output_path="data/corpus/cbio_disease/curated_single_disease_NCIT_sampled_with_official_label.csv"
)


03/07/2025 02:59:14 PM - INFO - NCIDb: Fetching concept data for 134 codes in batches of 50
03/07/2025 02:59:19 PM - INFO - NCIDb: Processed batch 1 of 3
03/07/2025 02:59:19 PM - INFO - NCIDb: Processed batch 2 of 3
03/07/2025 02:59:21 PM - INFO - NCIDb: Processed batch 3 of 3
03/07/2025 02:59:21 PM - INFO - NCIDb: Retrieved 134 concept data entries
03/07/2025 02:59:21 PM - INFO - DefaultLogger: 🟡 Found 8 mismatches out of 134 (5.97%)
03/07/2025 02:59:21 PM - INFO - DefaultLogger: ✅ Saved with 'official_label' to data/corpus/cbio_disease/curated_single_disease_NCIT_sampled_with_official_label.csv


test run

In [16]:
df = pd.read_csv("data/corpus/cbio_disease/curated_single_disease_NCIT_sampled_with_official_label.csv")
large_corpus = pd.read_csv(
    'data/corpus/cbio_disease/disease_corpus_updated.csv')

query_list = df['original_cancer_type_value'].tolist()
small_corpus_list = df['official_label'].tolist()
large_corpus_list = large_corpus['official_label'].tolist()

cura_map = dict(zip(df['original_cancer_type_value'], df['official_label']))

In [17]:
# LM Sapbert
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='sap-bert',
                                      category='disease',
                                      topk=5,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='lm',
                                      **other_params)
lm_sapbert_result = onto_engine_large.run()

07/07/2025 06:41:42 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
07/07/2025 06:41:42 PM - INFO - OntoMapEngine: Running Ontology Mapping
07/07/2025 06:41:42 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
07/07/2025 06:41:42 PM - INFO - OntoMapEngine: Replacing shortNames using rule-based name mapping
07/07/2025 06:41:42 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module


No sentence-transformers model found with name model_cache/sap-bert. Creating a new one with mean pooling.


07/07/2025 06:41:48 PM - INFO - FAISSSQLiteSearch: 13924 new terms to add to the index.


Batches: 100%|██████████| 7/7 [00:00<00:00, 73.67it/s]


07/07/2025 06:42:18 PM - INFO - OntoMapLM: True - Vector store initialized for method=sap-bert, category=disease, om_strategy=lm


In [18]:
lm_sapbert_accuracy_df = calc.calc_accuracy(lm_sapbert_result)
print(lm_sapbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  79.850746
1  Top 3 Matches  86.567164
2  Top 5 Matches  89.552239


In [ ]:
lm_sapbert_result

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score,updated_value,top1_accuracy,top3_accuracy,top5_accuracy
0,ACUTE MYELOID LEUKEMIA,Acute Myeloid Leukemia,1,2,Acute Myeloid Leukemia,0.9830,"Acute Myeloid Leukemia, Not Otherwise Specified",0.9319,Acute Leukemia,0.8983,Myeloid Leukemia,0.8529,Adult Acute Myeloid Leukemia,0.8357,ACUTE MYELOID LEUKEMIA,True,True,True
1,ADENOCARCINOMA OF GALLBLADDER,Gallbladder Adenocarcinoma,1,2,Gallbladder Adenocarcinoma,0.9601,Gallbladder Carcinoma,0.8857,"Gallbladder Adenocarcinoma, Biliary Type",0.8653,Gallbladder Clear Cell Adenocarcinoma,0.8468,Gallbladder Hepatoid Adenocarcinoma,0.8304,ADENOCARCINOMA OF GALLBLADDER,True,True,True
2,ADENOSARCOMA,Adenosarcoma,1,2,Adenosarcoma,0.9729,Angiosarcoma,0.7545,Sarcomatoid Carcinoma,0.7387,Chondrosarcoma,0.7237,Adult Angiosarcoma,0.7236,ADENOSARCOMA,True,True,True
3,ADRENAL TUMOR,Adrenal Gland Neoplasm,1,2,Adrenal Gland Neoplasm,0.9529,Adrenal Cortical Neoplasm,0.8611,Adrenal Medulla Neoplasm,0.8081,Adrenal Rest Tumor,0.7481,Benign Adrenal Gland Neoplasm,0.7429,ADRENAL TUMOR,True,True,True
4,ADRENOCORTICAL ADENOMA,Adrenal Cortical Adenoma,1,2,Adrenal Cortical Adenoma,0.9262,Androgen-Producing Adrenal Cortical Adenoma,0.8736,Cortisol-Producing Adrenal Cortical Adenoma,0.8637,Adrenal Cortical Adenomatoid Tumor,0.8529,Aldosterone-Producing Adrenal Cortical Adenoma,0.8489,ADRENOCORTICAL ADENOMA,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,UROTHELIAL CARCINOMA,Urothelial Carcinoma,1,2,Urothelial Carcinoma,0.9833,Bladder Urothelial Carcinoma,0.9178,Urachal Urothelial Carcinoma,0.8541,Invasive Urothelial Carcinoma,0.8533,Ureter Urothelial Carcinoma,0.8393,UROTHELIAL CARCINOMA,True,True,True
130,UTERINE ENDOMETRIOID CARCINOMA,Endometrial Endometrioid Adenocarcinoma,1,2,Endometrial Endometrioid Adenocarcinoma,0.9278,Endometrioid Adenocarcinoma,0.9203,Endometrioid Tumor,0.8931,Malignant Ovarian Endometrioid Tumor,0.8923,Cervical Endometrioid Adenocarcinoma,0.8896,UTERINE ENDOMETRIOID CARCINOMA,True,True,True
131,UTERINE SARCOMA,Uterine Corpus Sarcoma,1,2,Uterine Corpus Sarcoma,0.8674,Uterine Carcinosarcoma,0.8360,Malignant Uterine Neoplasm,0.7909,Uterine Corpus Undifferentiated Sarcoma,0.7752,Uterine Angiosarcoma,0.7672,UTERINE SARCOMA,True,True,True
132,VAGINAL CANCER,Vaginal Carcinoma,1,2,Vaginal Carcinoma,0.9284,Malignant Vaginal Neoplasm,0.9022,Vaginal Adenocarcinoma,0.8293,Vaginal Melanoma,0.7923,Vaginal Sarcoma,0.7913,VAGINAL CANCER,True,True,True


In [ ]:
unmatched_df = lm_sapbert_result[lm_sapbert_result["match_level"] == 99]
print(f"Number of unmatched queries: {unmatched_df.shape[0]}")
display(unmatched_df.head())

Number of unmatched queries: 14


,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score,updated_value,top1_accuracy,top3_accuracy,top5_accuracy
11,"BLADDER/URINARY TRACT CANCER, NOS",Malignant Urinary System Neoplasm,99,2,Urinary Tract Carcinoma,0.8503,Malignant Bladder Neoplasm,0.8381,Malignant Urinary Tract Neoplasm,0.8158,Bladder Carcinoma,0.8059,Urinary System Carcinoma,0.7772,"BLADDER/URINARY TRACT CANCER, NOS",False,False,False
13,BLOOD CANCER,Liquid Tumor,99,2,Leukemia,0.6711,Non-Hematologic Malignancy,0.6578,Malignant Bone Marrow Neoplasm,0.6561,Malignant Blood Vessel Neoplasm,0.6481,Erythroid Neoplasm,0.6345,BLOOD CANCER,False,False,False
22,CANCER OF UNKNOWN PRIMARY,Malignant Neoplasm,99,2,Carcinoma of Unknown Primary,0.9631,Malignant Neoplasm of Unknown Primary,0.9261,Adenocarcinoma of Unknown Primary,0.8689,Undifferentiated Carcinoma of Unknown Primary,0.8605,Melanoma of Unknown Primary,0.8381,CANCER OF UNKNOWN PRIMARY,False,False,False
24,CARCINOMA WITH SQUAMOUS DIFFERENTIATION,Gallbladder Squamous Cell Carcinoma,99,2,Squamous Cell Carcinoma,0.9147,Skin Squamous Cell Carcinoma,0.8184,Squamous Cell Neoplasm,0.8046,"Squamous Cell Carcinoma, Clear Cell Type",0.7739,Carcinoma Simplex,0.7679,CARCINOMA WITH SQUAMOUS DIFFERENTIATION,False,False,False
38,ESOPHAGOGASTRIC CANCER,NCI CTEP SDC Gastroesophageal Cancer Sub-Categ...,99,2,Esophageal and Gastroesophageal Junction Carci...,0.7981,Gastroesophageal Junction Carcinoma,0.7794,Gastroesophageal Junction Adenocarcinoma,0.7080,Esophageal Carcinoma,0.7018,Gastric Carcinoma,0.6912,ESOPHAGOGASTRIC CANCER,False,False,False


In [ ]:
lm_sapbert_result.to_csv("data/corpus/cbio_disease/lm_sapbert_disease_new_result.csv", index=False)

In [19]:
# ST Sapbert
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='mt-sap-bert',
                                      category='disease',
                                      topk=5,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='st',
                                      **other_params)
st_sapbert_result = onto_engine_large.run()

07/07/2025 09:17:07 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
07/07/2025 09:17:07 PM - INFO - OntoMapEngine: Running Ontology Mapping
07/07/2025 09:17:07 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
07/07/2025 09:17:07 PM - INFO - OntoMapEngine: Replacing shortNames using rule-based name mapping
07/07/2025 09:17:07 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module


No sentence-transformers model found with name model_cache/mt-sap-bert. Creating a new one with mean pooling.
No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext-mean-token. Creating a new one with mean pooling.


In [20]:
st_sapbert_accuracy_df = calc.calc_accuracy(st_sapbert_result)
print(st_sapbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  76.865672
1  Top 3 Matches  84.328358
2  Top 5 Matches  85.074627


In [22]:
unmatched_stsb_df = st_sapbert_result[st_sapbert_result["match_level"] == 99]
print(f"Number of unmatched queries: {unmatched_stsb_df.shape[0]}")
display(unmatched_stsb_df.head())

Number of unmatched queries: 20


,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score,updated_value,top1_accuracy,top3_accuracy,top5_accuracy
11,"BLADDER/URINARY TRACT CANCER, NOS",Malignant Urinary System Neoplasm,99,2,Urinary Tract Carcinoma,0.8018,Bladder Carcinoma,0.7718,Bladder Neoplasm,0.7703,Urinary System Carcinoma,0.7551,Urinary Tract Neoplasm,0.7412,"BLADDER/URINARY TRACT CANCER, NOS",False,False,False
13,BLOOD CANCER,Liquid Tumor,99,2,Malignant Blood Vessel Neoplasm,0.7235,Carcinoma,0.7106,Blood Vessel Neoplasm,0.7020,Carcinomatosis,0.6800,Malignant Neoplasm,0.6796,BLOOD CANCER,False,False,False
19,BREAST INVASIVE DUCTAL CARCINOMA,Invasive Breast Carcinoma of No Special Type,99,2,Breast Ductal Carcinoma,0.9054,Breast Ductal Carcinoma In Situ,0.8745,Invasive Breast Carcinoma,0.8687,Invasive Breast Ductal Carcinoma and Lobular C...,0.8460,Invasive Breast Ductal Carcinoma and Invasive ...,0.8318,BREAST INVASIVE DUCTAL CARCINOMA,False,False,False
22,CANCER OF UNKNOWN PRIMARY,Malignant Neoplasm,99,2,Carcinoma of Unknown Primary,0.9826,Undifferentiated Carcinoma of Unknown Primary,0.8883,Malignant Neoplasm of Unknown Primary,0.8704,Adenocarcinoma of Unknown Primary,0.8510,Melanoma of Unknown Primary,0.8200,CANCER OF UNKNOWN PRIMARY,False,False,False
24,CARCINOMA WITH SQUAMOUS DIFFERENTIATION,Gallbladder Squamous Cell Carcinoma,99,2,Squamous Cell Carcinoma,0.8952,Squamous Cell Neoplasm,0.8025,Skin Squamous Cell Carcinoma,0.7728,Advanced Squamous Cell Carcinoma,0.7687,Adenocarcinoma with Squamous Metaplasia,0.7486,CARCINOMA WITH SQUAMOUS DIFFERENTIATION,False,False,False


In [ ]:
# ST Sapbert
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='mt-sap-bert',
                                      category='disease',
                                      topk=5,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='st',
                                      **other_params)
st_sapbert_result = onto_engine_large.run()

In [26]:
# Produce a new file for testing

path = "data/corpus/cbio_disease/curated_single_disease_NCIT_sampled_with_official_label.csv"
df = pd.read_csv(path)

keep_cols = [
    "curation_id",
    "original_disease_value",
    "curated_disease_source",
    "curated_disease",
    "curated_disease_ontology_term_id",
    "original_cancer_subtype_value",
    "curated_cancer_subtype",
    "curated_cancer_subtype_ontology_term_id",
    "original_cancer_type_value",
    "curated_cancer_type",
    "curated_cancer_type_ontology_term_id",
    "original_cancer_type_detailed_value",
    "curated_cancer_type_detailed",
    "curated_cancer_type_detailed_ontology_term_id",
    "official_label"
]

df = df[keep_cols]

df['sampleId'] = df['curation_id'].str.split(':').str[-1]

info_df = pd.read_csv("data/non_na_ratio_0.3.csv")
info_df = info_df[['sampleId', 'studyId', 'ONCOTREE_CODE', 'PRIMARY_SITE']]

merged = pd.merge(df, info_df, on='sampleId', how='left')

print(merged.head())
merged.to_csv("data/corpus/cbio_disease/query_with_selected_fields.csv", index=False)

                             curation_id original_disease_value  \
0  acc_tcga:TCGA-OR-A5J1:TCGA-OR-A5J1-01                    NaN   
1  acc_tcga:TCGA-OR-A5J1:TCGA-OR-A5J1-01                    NaN   
2      all_stjude_2015:SJINF001:SJINF001                    NaN   
3  all_stjude_2013:SJHYPO001:SJHYPO001-D                    NaN   
4          acbc_mskcc_2015:AdCC5T:AdCC5T                    NaN   

  curated_disease_source curated_disease curated_disease_ontology_term_id  \
0                    NaN             NaN                              NaN   
1                    NaN             NaN                              NaN   
2                    NaN             NaN                              NaN   
3                    NaN             NaN                              NaN   
4                    NaN             NaN                              NaN   

  original_cancer_subtype_value curated_cancer_subtype  \
0                           NaN                    NaN   
1                 

In [ ]:
# This script retrieves a service ticket from the UMLS API and searches for a specific term.
# We can find that the term is not found in the NCI subset of UMLS, even though we retrieved the term from the NCI website.

import requests

API_KEY = "07bb3180-6b91-44a7-8fc4-ca051b1bacb2"
TERM = "Cutaneous Compound Spitz Nevus" 
SERVICE = "http://umlsks.nlm.nih.gov"

def get_tgt(api_key):
    response = requests.post(
        "https://utslogin.nlm.nih.gov/cas/v1/api-key",
        data={"apikey": api_key}
    )
    if response.status_code == 201:
        tgt_url = response.headers["location"]
        return tgt_url
    else:
        print("Failed to get TGT:", response.text)
        return None

def get_service_ticket(tgt_url):
    response = requests.post(
        tgt_url,
        data={"service": SERVICE}
    )
    if response.status_code == 200:
        return response.text
    else:
        print("Failed to get service ticket:", response.text)
        return None

def search_term(term, ticket):
    url = "https://uts-ws.nlm.nih.gov/rest/search/current"
    params = {
        "string": term,
        "searchType": "exact",
        "sabs": "NCI",
        "returnIdType": "code",
        "ticket": ticket
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json()
        items = results.get("result", {}).get("results", [])
        for item in items:
            print(f"Name: {item.get('name')}, UI: {item.get('ui')}, RootSource: {item.get('rootSource')}")
        if not items:
            print("No results found.")
    else:
        print("Search failed:", response.text)

if __name__ == "__main__":
    tgt = get_tgt(API_KEY)
    if tgt:
        st = get_service_ticket(tgt)
        if st:
            print(f"Searching for term: {TERM}")
            search_term(TERM, st)


Searching for term: Cutaneous Compound Spitz Nevus
No results found.


Change the rag to use ncit code rather than term -> code retrival

Disease

In [27]:
df = pd.read_csv('data/corpus/cbio_disease/disease_query_updated.csv')
large_corpus = pd.read_csv(
    'data/corpus/cbio_disease/disease_corpus_updated.csv')

query_list = df['original_value'].tolist()
small_corpus_list = df['curated_ontology'].tolist()
large_corpus_list = large_corpus['official_label'].tolist()

cura_map = dict(zip(df['original_value'], df['curated_ontology']))

In [24]:
# ST Strategy w/ SapBERT-mean-token
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='mt-sap-bert',
                                      category='disease',
                                      topk=5,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='st',
                                      **other_params)
st_sapbert_result = onto_engine_large.run()

07/07/2025 03:04:38 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
07/07/2025 03:04:38 AM - INFO - OntoMapEngine: Running Ontology Mapping
07/07/2025 03:04:38 AM - INFO - OntoMapEngine: Separating exact and non-exact matches
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replacing shortNames using rule-based name mapping
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replaced: ACC → Adrenocortical Carcinoma
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replaced: ACYC → Adenoid Cystic Carcinoma
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replaced: AML → Acute Myeloid Leukemia
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replaced: ANSC → Anal Squamous Cell Carcinoma
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replaced: APAD → Appendiceal Adenocarcinoma
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replaced: ASPS → Alveolar Soft Part Sarcoma
07/07/2025 03:04:39 AM - INFO - OntoMapEngine: Replaced: ATRT → Atypical Teratoid/Rhabdoid Tumor
07/07/2025 03:04:39 AM - INF

Batches: 100%|██████████| 7/7 [00:00<00:00, 80.86it/s]
No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext-mean-token. Creating a new one with mean pooling.


In [26]:
st_sapbert_result.to_csv("data/outputs/2025/large_corpus/0708/st_sapbert_disease.csv", index=False)

In [25]:
st_sapbert_accuracy_df = calc.calc_accuracy(st_sapbert_result)
print(st_sapbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  75.370729
1  Top 3 Matches  84.655061
2  Top 5 Matches  87.685364


In [8]:
# rag Strategy w/ PubMed-BERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='pubmed-bert',
                                      category='disease',
                                      topk=5,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='rag',
                                      corpus_df=large_corpus,
                                      **other_params)
pubmed_bert_result = onto_engine_large.run()

07/07/2025 03:51:47 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
07/07/2025 03:51:47 AM - INFO - OntoMapEngine: Running Ontology Mapping
07/07/2025 03:51:47 AM - INFO - OntoMapEngine: Separating exact and non-exact matches
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replacing shortNames using rule-based name mapping
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replaced: ACC → Adrenocortical Carcinoma
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replaced: ACYC → Adenoid Cystic Carcinoma
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replaced: AML → Acute Myeloid Leukemia
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replaced: ANSC → Anal Squamous Cell Carcinoma
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replaced: APAD → Appendiceal Adenocarcinoma
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replaced: ASPS → Alveolar Soft Part Sarcoma
07/07/2025 03:51:48 AM - INFO - OntoMapEngine: Replaced: ATRT → Atypical Teratoid/Rhabdoid Tumor
07/07/2025 03:51:48 AM - INF

Processing queries:   0%|          | 0/1551 [00:00<?, ?it/s]

07/07/2025 03:51:49 AM - INFO - FAISSSQLiteSearch: 13927 new terms to add to the index.
07/07/2025 03:51:49 AM - INFO - FAISSSQLiteSearch: Using provided DataFrame to update term-code pairs.
07/07/2025 03:51:49 AM - INFO - FAISSSQLiteSearch: Using provided DataFrame to fetch term-code pairs.
07/07/2025 03:51:50 AM - INFO - FAISSSQLiteSearch: Retrieved codes for 13924 terms
07/07/2025 03:51:50 AM - INFO - FAISSSQLiteSearch: Fetching concept data for 13927 unique codes
07/07/2025 03:51:50 AM - INFO - NCIDb: Fetching concept data for 13927 codes in batches of 50
07/07/2025 03:51:54 AM - INFO - NCIDb: Processed batch 1 of 279
07/07/2025 03:51:54 AM - INFO - NCIDb: Processed batch 2 of 279
07/07/2025 03:51:57 AM - INFO - NCIDb: Processed batch 3 of 279
07/07/2025 03:52:00 AM - INFO - NCIDb: Processed batch 4 of 279
07/07/2025 03:52:03 AM - INFO - NCIDb: Processed batch 5 of 279
07/07/2025 03:52:05 AM - INFO - NCIDb: Processed batch 6 of 279
07/07/2025 03:52:11 AM - INFO - NCIDb: Processed b

Building context and records: 100%|██████████| 13924/13924 [00:01<00:00, 12908.44it/s]

07/07/2025 04:04:44 AM - INFO - FAISSSQLiteSearch: Inserting 13927 records into SQLite


07/07/2025 04:04:44 AM - INFO - FAISSSQLiteSearch: Starting vector embedding and FAISS index insertion


























Batches: 100%|██████████| 32/32 [00:04<00:00,  7.49it/s]
























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.29it/s]


























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.81it/s]
























Batches: 100%|██████████| 32/32 [00:03<00:00, 10.51it/s]





















Batches: 100%|██████████| 32/32 [00:02<00:00, 11.31it/s]





















Batches: 100%|██████████| 32/32 [00:02<00:00, 12.10it/s]





























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.25it/s]





















Batches: 100%|██████████| 32/32 [00:02<00:00, 11.15it/s]























Batches: 100%|██████████| 32/32 [00:03<00:00, 10.54it/s]




























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.65it/s]
























Batches: 100%|██████████| 32/32 [00:03<00:00, 10.57it/s]























Batches: 100%|██████████| 32/32 [00:02<00:00, 10.75it/s]























Batch

07/07/2025 04:06:14 AM - INFO - FAISSSQLiteSearch: Finished fetching and storing all terms.
07/07/2025 04:06:14 AM - INFO - OntoMapRAG: True - Vector store initialized for method=pubmed-bert, category=disease, om_strategy=rag



Processing queries: 100%|██████████| 1551/1551 [15:03<00:00,  1.72it/s]


07/07/2025 04:06:51 AM - INFO - OntoMapRAG: Results Generated


In [21]:
rag_pubmed_accuracy_df = calc.calc_accuracy(pubmed_bert_result)
print(rag_pubmed_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  57.704707
1  Top 3 Matches  78.078659
2  Top 5 Matches  83.816892


In [22]:
pubmed_bert_result.to_csv("rag_pubmed_bert_disease_new.csv", index=False)

use new test df

In [6]:
df = pd.read_csv('data/corpus/cbio_disease/query_with_selected_fields.csv')
large_corpus = pd.read_csv(
    'data/corpus/cbio_disease/disease_corpus_updated.csv')

query_list = df['original_cancer_type_value'].tolist()
large_corpus_list = large_corpus['official_label'].tolist()

cura_map = dict(zip(df['original_cancer_type_value'], df['curated_cancer_type']))

In [ ]:
# bie Strategy w/ sapbert-mean-token
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='sap-bert',
                                      category='disease',
                                      topk=5,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='bie',
                                        corpus_df=large_corpus,
                                        query_df=df,
                                      **other_params)
sapbert_bie_result = onto_engine_large.run()

In [28]:
# rag Strategy w/ PubMed-BERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='pubmed-bert',
                                      category='disease_new',
                                      topk=20,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='rag',
                                      **other_params)
pubmed_bert_result = onto_engine_large.run()

01/07/2025 06:08:38 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
01/07/2025 06:08:38 PM - INFO - OntoMapEngine: Running Ontology Mapping
01/07/2025 06:08:38 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
01/07/2025 06:08:38 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
01/07/2025 06:08:38 PM - INFO - OntoMapRAG: Initialized OntoMapRAG module
01/07/2025 06:08:38 PM - INFO - OntoMapRAG: Generating results table


Processing queries:   0%|          | 0/1551 [00:00<?, ?it/s]

01/07/2025 06:08:40 PM - INFO - OntoMapRAG: True - Vector store initialized for method=pubmed-bert, category=disease_new, om_strategy=rag


Processing queries: 100%|██████████| 1551/1551 [00:40<00:00, 38.20it/s]


01/07/2025 06:09:19 PM - INFO - OntoMapRAG: Results Generated


In [29]:
pubmed_bert_result

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,...,top16_match,top16_score,top17_match,top17_score,top18_match,top18_score,top19_match,top19_score,top20_match,top20_score
0,25-30% DUCTAL COMPONENT,Prostate Ductal Adenocarcinoma,99,2,Breast Ductal Carcinoma,0.9188,Breast Mixed Ductal and Lobular Carcinoma,0.9157,"Breast Ductal Carcinoma In Situ, Intermediate ...",0.9115,...,"Breast Ductal Carcinoma In Situ, Non-Comedo Type",0.9052,Salivary Gland Intercalated Duct Adenoma,0.9049,Breast Ductal Hyperplasia,0.9047,Invasive Breast Ductal Carcinoma and Lobular C...,0.9046,Pancreatic Poorly Differentiated Ductal Adenoc...,0.9045
1,82463 NEUROENDOCRINE CARCINOMA NOS,Neuroendocrine Carcinoma,1,2,Neuroendocrine Carcinoma,0.9301,Breast Small Cell Neuroendocrine Carcinoma,0.9298,Breast Solid Neuroendocrine Carcinoma,0.9292,...,Esophageal Neuroendocrine Carcinoma,0.9246,Breast Neuroendocrine Tumor,0.9246,Head and Neck Large Cell Neuroendocrine Carcinoma,0.9242,Paranasal Sinus Small Cell Neuroendocrine Carc...,0.9240,Sinonasal Neuroendocrine Carcinoma,0.9239
2,AB,Thymoma Type AB,3,2,Acute Promyelocytic Leukemia with BCOR-RARA,0.8746,Adult B Acute Lymphoblastic Leukemia with t(9;...,0.8737,Thymoma Type AB,0.8723,...,Stage IB Bone Cancer AJCC v8,0.8687,Stage IIB Bone Cancer AJCC v7,0.8686,Stage IB Bone Cancer AJCC v7,0.8685,Stage IIB Anal Cancer AJCC v8,0.8684,B Acute Lymphoblastic Leukemia with t(9;22)(q3...,0.8684
3,ACANTHOLYTIC,Skin Acantholytic Squamous Cell Carcinoma,4,2,Acantholytic Acanthoma,0.9320,Epidermolytic Acanthoma,0.9174,Acanthoma,0.9163,...,Vulvar Seborrheic Keratosis,0.8816,Inverted Follicular Keratosis,0.8813,Syringoacanthoma,0.8802,Vulvar Inverted Follicular Keratosis,0.8795,External Ear Actinic Keratosis,0.8791
4,ACC,Adrenal Cortical Carcinoma,14,2,Stage II Adrenal Cortical Carcinoma AJCC v7,0.8783,Stage III Adrenal Cortical Carcinoma AJCC v7,0.8775,Stage IV Adrenal Cortical Carcinoma AJCC v7,0.8773,...,Adult Anaplastic Astrocytoma,0.8722,Advanced Breast Carcinoma,0.8722,Stage IV Adrenal Cortical Carcinoma AJCC v8,0.8721,Advanced Cholangiocarcinoma,0.8720,Advanced Cervical Adenosquamous Carcinoma,0.8716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,WILMS' TUMOR,Wilms Tumor,1,2,Wilms Tumor,0.9441,Kidney Wilms Tumor,0.9403,Adult Kidney Wilms Tumor,0.9385,...,Anaplastic Kidney Wilms Tumor,0.9247,Stage III Kidney Wilms Tumor,0.9234,Refractory Wilms Tumor,0.9233,Stromal Predominant Kidney Wilms Tumor,0.9232,Metachronous Kidney Wilms Tumor,0.9230
1547,WILMS' TUMORS,Wilms Tumor,1,2,Wilms Tumor,0.9449,Kidney Wilms Tumor,0.9422,Adult Kidney Wilms Tumor,0.9401,...,Anaplastic Kidney Wilms Tumor,0.9265,Stromal Predominant Kidney Wilms Tumor,0.9252,Metachronous Kidney Wilms Tumor,0.9250,Stage III Kidney Wilms Tumor,0.9248,Refractory Wilms Tumor,0.9240
1548,WITH DUCTAL AND LOBULAR PHENOTYPES,Breast Mixed Ductal and Lobular Carcinoma,1,2,Breast Mixed Ductal and Lobular Carcinoma,0.9276,Invasive Breast Ductal Carcinoma and Lobular C...,0.9251,Breast Ductal Carcinoma In Situ and Lobular Ca...,0.9249,...,Breast Lobular Carcinoma In Situ,0.9178,"Invasive Breast Lobular Carcinoma, Solid Variant",0.9171,Invasive Breast Lobular Carcinoma with Predomi...,0.9169,Breast Lobular Neoplasia,0.9162,"Invasive Breast Lobular Carcinoma, Tubulolobul...",0.9155
1549,WT,Wilms Tumor,2,2,ST2 Diffuse Large B-Cell Lymphoma,0.8712,Wilms Tumor,0.8675,"Diffuse Low Grade Glioma, FGFR1-Mutant",0.8663,...,Myelodysplastic Syndrome/Acute Myeloid Leukemi...,0.8623,"Diffuse Hemispheric Glioma, H3 G34-Mutant",0.8622,Tongue Verrucous Carcinoma,0.8621,"Diffuse Low Grade Glioma, MAPK Pathway-Altered",0.8620,Kidney Wilms Tumor,0.8618


In [30]:
pubmed_bert_result.to_csv("rag_pubmed_bert_disease_new_20.csv", index=False)

In [25]:
# st Strategy w/ PubMed-BERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='mt-sap-bert',
                                      category='disease_new',
                                      topk=20,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='st',
                                      **other_params)
sb_result_20 = onto_engine_large.run()

01/07/2025 02:12:39 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
01/07/2025 02:12:39 PM - INFO - OntoMapEngine: Running Ontology Mapping
01/07/2025 02:12:39 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
01/07/2025 02:12:40 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
01/07/2025 02:12:40 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module


No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext-mean-token. Creating a new one with mean pooling.


In [26]:
sb_result_20.to_csv("st_sapbert_diseae_20.csv", index=False)

In [27]:
sb_result_20

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,...,top16_match,top16_score,top17_match,top17_score,top18_match,top18_score,top19_match,top19_score,top20_match,top20_score
0,25-30% DUCTAL COMPONENT,Prostate Ductal Adenocarcinoma,9,2,Ductal Eccrine Adenocarcinoma,0.6203,Oligometastatic Pancreatic Ductal Adenocarcinoma,0.6008,Ductal Eccrine Carcinoma with Abundant Fibromy...,0.5990,...,Lipid-Rich Carcinoma,0.5775,Dermal Duct Tumor,0.5748,Distal Cholangiocarcinoma,0.5740,Pancreatic Moderately Differentiated Ductal Ad...,0.5728,Adenocarcinoma with Apocrine Metaplasia,0.5725
1,82463 NEUROENDOCRINE CARCINOMA NOS,Neuroendocrine Carcinoma,1,2,Neuroendocrine Carcinoma,0.7585,Cervical Neuroendocrine Carcinoma,0.7151,Advanced Neuroendocrine Carcinoma,0.7141,...,Colon Neuroendocrine Carcinoma,0.6756,Grade II Neuroendocrine Carcinoma,0.6752,Metastatic Neuroendocrine Carcinoma,0.6731,Gastroesophageal Junction Neuroendocrine Carci...,0.6725,Neuroendocrine Neoplasm,0.6670
2,AB,Thymoma Type AB,8,2,Branchioma,0.6178,Blastoma,0.6003,Adult Acute Basophilic Leukemia,0.5979,...,Neuroblastoma,0.5599,Adenocarcinoma,0.5583,Breast Atypical Ductal Hyperplasia,0.5576,Advanced Breast Adenocarcinoma,0.5571,Atypical Adenoma,0.5567
3,ACANTHOLYTIC,Skin Acantholytic Squamous Cell Carcinoma,9,2,Acantholytic Acanthoma,0.8033,Acanthoma,0.7408,Epidermolytic Acanthoma,0.7229,...,Basal Cell Adenoma,0.5783,Lymphoma,0.5750,Secretory Meningioma,0.5699,AH Amyloidosis,0.5675,Inverted Follicular Keratosis,0.5675
4,ACC,Adrenal Cortical Carcinoma,99,2,Adenoma,0.6409,Adult Cholangiocarcinoma,0.6319,Advanced Cholangiocarcinoma,0.6314,...,Acinar Cell Carcinoma,0.6017,Locally Advanced Cholangiocarcinoma,0.6014,Occult Lung Adenocarcinoma,0.5985,Anal Adenocarcinoma,0.5962,Cerebral Astrocytoma,0.5948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,WILMS' TUMOR,Wilms Tumor,1,2,Wilms Tumor,0.9868,Kidney Wilms Tumor,0.9630,Childhood Kidney Wilms Tumor,0.9016,...,Blastema Predominant Kidney Wilms Tumor,0.7650,Stage I Kidney Wilms Tumor,0.7430,Stage IV Kidney Wilms Tumor,0.7365,Stage V Kidney Wilms Tumor,0.7349,Metachronous Kidney Wilms Tumor,0.7337
1547,WILMS' TUMORS,Wilms Tumor,1,2,Wilms Tumor,0.9844,Kidney Wilms Tumor,0.9605,Childhood Kidney Wilms Tumor,0.8999,...,Refractory Wilms Tumor,0.7620,Stage I Kidney Wilms Tumor,0.7393,Metachronous Kidney Wilms Tumor,0.7334,Stage IV Kidney Wilms Tumor,0.7325,Recurrent Kidney Wilms Tumor,0.7314
1548,WITH DUCTAL AND LOBULAR PHENOTYPES,Breast Mixed Ductal and Lobular Carcinoma,1,2,Breast Mixed Ductal and Lobular Carcinoma,0.6855,Diffuse Lipomatosis,0.6514,Diffuse Intraductal Papillomatosis,0.6390,...,Diffuse Type Adenocarcinoma,0.5890,Intraductal Papillomatosis,0.5861,Breast Atypical Ductal Hyperplasia,0.5847,Follicular Lymphoma with Predominantly Diffuse...,0.5839,Invasive Breast Ductal Carcinoma and Invasive ...,0.5825
1549,WT,Wilms Tumor,1,2,Wilms Tumor,0.6092,Well Differentiated Malignant Neoplasm,0.5959,Nonanaplastic Kidney Wilms Tumor,0.5926,...,Favorable Hodgkin Lymphoma,0.5607,Favorable Non-Hodgkin Lymphoma,0.5595,Prostate Wilms Tumor,0.5593,Polyposis,0.5584,Cervical Wilms Tumor,0.5557


In [23]:
# rag Strategy w/ PubMed-BERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='mt-sap-bert',
                                      category='disease_new',
                                      topk=10,
                                      query=query_list,
                                      corpus=large_corpus_list,
                                      cura_map=cura_map,
                                      om_strategy='rag',
                                      **other_params)
pubmed_bert_result2 = onto_engine_large.run()

01/07/2025 12:22:52 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
01/07/2025 12:22:52 PM - INFO - OntoMapEngine: Running Ontology Mapping
01/07/2025 12:22:52 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
01/07/2025 12:22:52 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
01/07/2025 12:22:52 PM - INFO - OntoMapRAG: Initialized OntoMapRAG module
01/07/2025 12:22:52 PM - INFO - OntoMapRAG: Generating results table


Processing queries:   0%|          | 0/1551 [00:00<?, ?it/s]

01/07/2025 12:22:52 PM - INFO - FAISSSQLiteSearch: Fetching NCI codes for 13924 new terms
01/07/2025 12:22:55 PM - INFO - FAISSSQLiteSearch: Processed UMLS batch 1 of 233
01/07/2025 12:22:58 PM - INFO - FAISSSQLiteSearch: Processed UMLS batch 2 of 233
01/07/2025 12:22:58 PM - INFO - UMLSDb: No NCI code found for term 'Von Hippel Lindau Syndrome-Associated Uterine Ligament Papillary Cystadenoma'
01/07/2025 12:23:01 PM - INFO - FAISSSQLiteSearch: Processed UMLS batch 3 of 233
01/07/2025 12:23:04 PM - INFO - UMLSDb: No NCI code found for term 'Vaginal Nevus'
01/07/2025 12:23:05 PM - INFO - FAISSSQLiteSearch: Processed UMLS batch 4 of 233
01/07/2025 12:23:05 PM - INFO - UMLSDb: No NCI code found for term 'Vulvar Angiomyxoma'
01/07/2025 12:23:05 PM - INFO - UMLSDb: No NCI code found for term 'Vulvar Acquired Nevus'
01/07/2025 12:23:08 PM - INFO - FAISSSQLiteSearch: Processed UMLS batch 5 of 233
01/07/2025 12:23:11 PM - INFO - FAISSSQLiteSearch: Processed UMLS batch 6 of 233
01/07/2025 12:23

Building context and records: 100%|██████████| 13720/13720 [00:01<00:00, 11656.25it/s]

01/07/2025 12:47:55 PM - INFO - FAISSSQLiteSearch: Inserting 13712 records into SQLite


01/07/2025 12:47:55 PM - INFO - FAISSSQLiteSearch: Starting vector embedding and FAISS index insertion



















Batches: 100%|██████████| 32/32 [00:02<00:00, 12.39it/s]

























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.13it/s]


























Batches: 100%|██████████| 32/32 [00:03<00:00,  8.73it/s]
























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.46it/s]





















Batches: 100%|██████████| 32/32 [00:02<00:00, 10.87it/s]





















Batches: 100%|██████████| 32/32 [00:02<00:00, 11.03it/s]



























Batches: 100%|██████████| 32/32 [00:03<00:00,  8.40it/s]





















Batches: 100%|██████████| 32/32 [00:02<00:00, 10.94it/s]

























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.73it/s]




















Batches: 100%|██████████| 32/32 [00:03<00:00, 10.66it/s]

























Batches: 100%|██████████| 32/32 [00:03<00:00,  9.46it/s]





















Batches: 100%|██████████| 32/32 [00:02<00:00, 10.67it/s]


























Batches: 100%|███

01/07/2025 12:49:30 PM - INFO - FAISSSQLiteSearch: Finished fetching and storing all terms.
01/07/2025 12:49:30 PM - INFO - OntoMapRAG: True - Vector store initialized for method=mt-sap-bert, category=disease_new, om_strategy=rag



Processing queries: 100%|██████████| 1551/1551 [27:12<00:00,  1.05s/it]


01/07/2025 12:50:04 PM - INFO - OntoMapRAG: Results Generated


In [24]:
rag_pubmed_accuracy_df = calc.calc_accuracy(pubmed_bert_result2)
print(rag_pubmed_accuracy_df) 

  Accuracy Level   Accuracy
0    Top 1 Match  24.242424
1  Top 3 Matches  44.165055
2  Top 5 Matches  55.254674


In [32]:
import pandas as pd

df = pd.read_csv("data/corpus/cbio_disease/curated_single_disease_NCIT_sampled.csv")

# Extract studyId from curation_id
df['studyId'] = df['curation_id'].apply(lambda x: str(x).split(":")[0])

# Count unique studyId
unique_study_ids = df['studyId'].nunique()
print(f"Number of unique studyId: {unique_study_ids}")
print(df['studyId'].unique())


Number of unique studyId: 43
['acc_tcga' 'all_stjude_2015' 'all_stjude_2013' 'acbc_mskcc_2015'
 'acyc_fmi_2014' 'aml_ohsu_2018' 'blca_tcga' 'bcc_unige_2016'
 'ampca_bcm_2016' 'angs_project_painter_2018' 'bfn_duke_nus_2015'
 'brain_cptac_2020' 'cesc_tcga' 'chol_tcga' 'cellline_ccle_broad'
 'ccle_broad_2019' 'brca_mapk_hp_msk_2021' 'gbm_tcga_pub'
 'histiocytosis_cobi_msk_2019' 'gct_msk_2020'
 'kirc_tcga_pan_can_atlas_2018' 'lung_msk_pdx'
 'lymphoma_cellline_msk_2020' 'meso_tcga_pan_can_atlas_2018'
 'mixed_pipseq_2017' 'mixed_allen_2018'
 'metastatic_solid_tumors_mich_2017' 'ntrk_msk_2019' 'msk_ch_2020'
 'msk_ch_ped_2021' 'pcpg_tcga_pub' 'ov_tcga_pan_can_atlas_2018'
 'prad_cdk12_mskcc_2020' 'pan_origimed_2020' 'pancan_pcawg_2020'
 'summit_2018' 'tgct_tcga_pan_can_atlas_2018'
 'thym_tcga_pan_can_atlas_2018' 'uvm_tcga_pan_can_atlas_2018'
 'gbc_mskcc_2022' 'luad_mskimpact_2021' 'msk_impact_2017'
 'lung_msk_mind_2020']


In [4]:
import pandas as pd
import asyncio
import httpx
from typing import Dict
from aiolimiter import AsyncLimiter

UMLS_API_KEY = "07bb3180-6b91-44a7-8fc4-ca051b1bacb2" 
CSV_PATH = "EDA/cancer_type_mapping.csv"
OUTPUT_PATH = "EDA/cancer_type_mapping_with_code.csv"

rate_limiter = AsyncLimiter(18, 1)  # 18 requests per second

async def get_nci_code(term: str, client: httpx.AsyncClient) -> str:
    url = "https://uts-ws.nlm.nih.gov/rest/search/current"
    params = {
        "string": term,
        "apiKey": UMLS_API_KEY,
        "pageNumber": 1,
        "searchType": "exact",
        "sabs": "NCI",
        "returnIdType": "code",
    }
    async with rate_limiter:
        try:
            resp = await client.get(url, params=params, timeout=10)
            resp.raise_for_status()
            results = resp.json().get("result", {}).get("results", [])
            for res in results:
                if res.get("rootSource") == "NCI":
                    return res.get("ui")
        except Exception as e:
            print(f"Error for {term}: {e}")
    return ""

async def main():
    df = pd.read_csv(CSV_PATH)
    terms = df["name"].dropna().unique().tolist()

    async with httpx.AsyncClient() as client:
        tasks = [get_nci_code(term, client) for term in terms]
        codes = await asyncio.gather(*tasks)

    term_to_code: Dict[str, str] = dict(zip(terms, codes))
    df["ncit_code"] = df["name"].map(term_to_code)
    df.to_csv(OUTPUT_PATH, index=False)
    print(f"✅ Output saved to: {OUTPUT_PATH}")

if __name__ == "__main__":
    asyncio.run(main())


✅ Output saved to: EDA/cancer_type_mapping_with_code.csv
